In [ ]:
! pip install mysql-connector-python

In [ ]:
from tkinter import *
from tkinter import messagebox
import re
import mysql.connector

In [ ]:
# Connecting to the MySQL server
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",)
mycursor = mydb.cursor(buffered=True)

In [ ]:
# Creating a function to create a database and table:
def DBCreation():
    # Creating a Database ["applicationDB"]
    mycursor.execute("CREATE DATABASE IF NOT EXISTS applicationDB")

    # Creating a Table [data]
    mycursor.execute("CREATE TABLE IF NOT EXISTS applicationDB.Data(\
        First_Name VARCHAR(60) NOT NULL, \
        Last_Name VARCHAR(60) NOT NULL, \
        Gender VARCHAR(10) NOT NULL, \
        Date_of_Birth DATE NOT NULL, \
        Mobile_Number VARCHAR(10) NOT NULL, \
        Aadhar_Number VARCHAR(12) NOT NULL, \
        PAN_Number VARCHAR(10) NOT NULL, \
        Address VARCHAR(200) NOT NULL)")
    
DBCreation()

In [ ]:
# Creating a class and necessary functions for the Insert Window
class insert_window(Toplevel):
    def __init__(self,parent):
        super().__init__(parent)
        self.title("Application Form")

        self.rowconfigure(list(range(0,11)), minsize = 50)
        self.columnconfigure(list(range(0,3)), minsize = 400)
        self.gender = IntVar()

        # Creating necessary labels
        instruction_label = Label(master = self, text = "Please enter the details", font = ("Calibri", 15))
        first_name_label = Label(master = self, text = "First Name* ")
        last_name_label = Label(master = self, text = "Last Name* ")
        gender_label = Label(master = self, text = "Gender* ")
        date_of_birth_label = Label(master = self, text = "Date of birth (YYYY-MM-DD)* ")
        mobile_label = Label(master = self, text = "Mobile* ")
        aadhar_number_label = Label(master = self, text = "Aadhar Number* ")
        pan_number_label = Label(master = self, text = "PAN Number* ")
        address_label = Label(master = self, text = "Address* ")

        # creating the entry boxes and radio button
        self.first_name_entry = Entry(master = self)
        self.last_name_entry = Entry(master = self)
        gender_frame = Frame(master = self)
        male_radio = Radiobutton(master = gender_frame, text = "Male", variable = self.gender, value = 1)
        male_radio.pack(side = "left")
        female_radio = Radiobutton(master = gender_frame, text = "female", variable = self.gender, value = 2)
        female_radio.pack(side = "right")
        self.date_of_birth_entry = Entry(master = self)
        self.mobile_entry = Entry(master = self)
        self.aadhar_number_entry = Entry(master = self)
        self.pan_number_entry = Entry(master = self)
        self.address_entry = Text(master = self, height = 7, width = 25)

        # Creating necessary buttons
        save_button = Button(master = self, text = "Save", width = 10, height = 2, command = self.save_clicked)
        clear_all_button = Button(master = self, text = "Clear all", width = 10, height = 2, command = self.clear_all)
        close_button = Button(master = self, text = "Close", width = 10, height = 2, command = self.destroy)

        # Placing the labels and buttons
        instruction_label.grid(row = 1, column = 1, sticky = "news")
        first_name_label.grid(row = 2, column = 0, sticky = 'w')
        last_name_label.grid(row = 3, column = 0, sticky = 'w')
        gender_label.grid(row = 4, column = 0, sticky = 'w')
        date_of_birth_label.grid(row = 5, column = 0, sticky = 'w')
        mobile_label.grid(row = 6, column = 0, sticky = 'w')
        aadhar_number_label.grid(row = 7, column = 0, sticky = 'w')
        pan_number_label.grid(row = 8, column = 0, sticky = 'w')
        address_label.grid(row = 9, column = 0, sticky = 'w')
        save_button.grid(row = 10, column = 1)
        clear_all_button.grid(row = 10, column = 1, sticky = 'e')
        close_button.grid(row = 10, column = 2, sticky = 'e')

        # Placing the entry boxes
        self.first_name_entry.grid(row = 2, column = 1)
        self.last_name_entry.grid(row = 3, column = 1)
        gender_frame.grid(row = 4, column = 1)
        self.date_of_birth_entry.grid(row = 5, column = 1)
        self.mobile_entry.grid(row = 6, column = 1)
        self.aadhar_number_entry.grid(row = 7, column = 1)
        self.pan_number_entry.grid(row = 8, column = 1)
        self.address_entry.grid(row = 9, column = 1)
    
    # Function to check the entered data before saving.
    def check_data(self):
        # Checking the First Name
        if self.first_name_entry.get() == '' or len(list(filter(lambda x:x.isdigit(),self.first_name_entry.get()))) > 0:
            messagebox.showerror("error", "First Name cannot be empty and can contain only alphabetical values")
            self.first_name_entry.delete(0, 'end')
            return False
        else:
            self.first_name = self.first_name_entry.get().strip().split(" ")
            self.first_name = " ".join(i.capitalize() for i in self.first_name)
        
        # Checking the Last Name
        if self.last_name_entry.get() == '' or len(list(filter(lambda x:x.isdigit(),self.last_name_entry.get()))) > 0:
            messagebox.showerror("error", "Last Name cannot be empty and can contain only alphabetical values")
            self.last_name_entry.delete(0, 'end')
            return False
        else:
            self.last_name = self.last_name_entry.get().strip().split(" ")
            self.last_name = " ".join(i.capitalize() for i in self.last_name)
        
        # Checking the Gender field
        if self.gender.get() == 1 :
            self.Gender = "Male"
        elif self.gender.get() == 2:
            self.Gender = "Female"
        else :
            messagebox.showerror("error", "Gender is a mandatory field")
            return False
        
        # Checking the Date of birth field
        self.date_of_birth = self.date_of_birth_entry.get()
        try: 
            self.dob = "-".join(list(re.findall(r"(\d{4})-(0[1-9]|1[0-2])-(0[1-9]|[12][0-9]|3[01])$", self.date_of_birth)[0]))
            if self.dob != self.date_of_birth:
                messagebox.showerror("error", "Please enter the date of birth in the given format")
                self.date_of_birth_entry.delete(0, 'end')
                return False
        except:
            messagebox.showerror("error", "Please enter the date of birth in the given format")
            self.date_of_birth_entry.delete(0, 'end')
            return False
        
        # Checking the mobile number
        self.mobile = self.mobile_entry.get()
        if len(self.mobile) != 10 or self.mobile == '' or len(list(filter(lambda i:i.isalpha(),self.mobile))) > 0:
            messagebox.showerror("error", "Mobile number cannot be empty and can only contain numerical values")
            self.mobile_entry.delete(0, 'end')
            return False
        
        # Checking Aadhar number
        self.aadhar_number = self.aadhar_number_entry.get()
        if self.aadhar_number == '' or len(self.aadhar_number) != 12 or len(list(filter(lambda i:i.isalpha(),self.aadhar_number))) > 0:
            messagebox.showerror("error", "Aadhar number cannot be empty and can only contain numerical values")
            self.aadhar_number_entry.delete(0, 'end')
            return False
        
        # Checking PAN Card
        self.pan_number = self.pan_number_entry.get()
        try:
            pan = re.findall(r"^[A-Z]{5}\d{4}[A-Z]{1}",self.pan_number)[0]
            if pan != self.pan_number:
                messagebox.showerror("error", "Please enter a valid PAN Number")
                self.pan_number_entry.delete(0, 'end')
                return False
        except:
            messagebox.showerror("error", "Please enter a valid PAN Number")
            self.pan_number_entry.delete(0, 'end')
            return False
        
        # Getting the address
        if len(self.address_entry.get(1.0, "end-1c")) == 0:
            messagebox.showerror("error", "Address cannot be empty")
            return False
        else:
            self.address = self.address_entry.get(1.0, "end-1c")
            

        # returning the collected data as a tuple
        self.data = (self.first_name, self.last_name, self.Gender, self.date_of_birth, self.mobile, self.aadhar_number, self.pan_number, self.address)
        return self.data
    
    # Function to clear all data from entry boxes
    def clear_all(self):
        self.first_name_entry.delete(0, 'end')
        self.last_name_entry.delete(0, 'end')
        self.gender.set(0)
        self.date_of_birth_entry.delete(0, 'end')
        self.mobile_entry.delete(0, 'end')
        self.aadhar_number_entry.delete(0, 'end')
        self.pan_number_entry.delete(0, 'end')
        self.address_entry.delete('1.0', 'end')

    # Creating a function to insert data
    def insert(self,data):
        # inserting data into table
        mycursor.execute(f"INSERT INTO applicationDB.Data (First_Name, \
            Last_Name, \
            Gender, \
            Date_of_Birth, \
            Mobile_Number, \
            Aadhar_Number, \
            PAN_Number, \
            Address) \
            VALUES {data}")
        mydb.commit()

    # Save button function
    def save_clicked(self):
        data_status = self.check_data()
        if data_status == False:
            print("Bad data found. Insert aborted")
        else:
            try:
                self.insert(data_status)
                messagebox.showinfo("showinfo", "Data Inserted successfully !")
                self.clear_all()
                print("Data inserted successfully")
            except Exception as error:
                print(error)

In [ ]:
class retrieve_window(Toplevel):
    def __init__(self,parent):
        super().__init__(parent)
        self.title("Search Data")
        self.rowconfigure(list(range(0,11)), minsize = 50)
        self.columnconfigure(list(range(0,3)), minsize=400)

        # Label creation
        instruction_label = Label(master = self, text = "Please enter the First and Last name of the person", font = ("Calibri", 15))
        first_name_label = Label(master = self, text = "First Name")
        last_name_label = Label(master = self, text = "Last Name")

        # Entry box creation
        self.first_name_entry = Entry(master = self)
        self.last_name_entry = Entry(master = self)

        # Button creation
        search_button = Button(master = self, text = "Search", width = 10, height = 2, command = self.search)
        clear_all_button = Button(master = self, text = "Clear all", width = 10, height = 2, command = self.clear_all)
        close_button = Button(master = self, text = "Close", width = 10, height = 2, command = self.destroy)

        # Label, button and entry box placements
        instruction_label.grid(row = 1, column = 1)
        first_name_label.grid(row = 2, column = 0, sticky = 'w')
        self.first_name_entry.grid(row = 2, column = 1)
        last_name_label.grid(row = 3, column = 0, sticky = 'w')
        self.last_name_entry.grid(row = 3, column = 1)
        search_button.grid(row = 4, column = 1)
        close_button.grid(row = 5, column = 2, sticky = 'e')
        clear_all_button.grid(row = 5, column = 1)

    # Check function
    def check(self):
        # Checking the First Name
        if len(list(filter(lambda x:x.isdigit(),self.first_name_entry.get()))) > 0:
            messagebox.showerror("error", "First Name can contain only alphabetical values")
            self.first_name_entry.delete(0, 'end')
            return False
        else:
            if len(self.first_name_entry.get()) > 0:
                self.first_name = self.first_name_entry.get().strip().split(" ")
                self.first_name = " ".join(i.capitalize() for i in self.first_name)
            else:
                self.first_name = ''
        # Checking the Last Name
        if len(list(filter(lambda x:x.isdigit(),self.last_name_entry.get()))) > 0:
            messagebox.showerror("error", "Last Name can contain only alphabetical values")
            self.last_name_entry.delete(0, 'end')
            return False
        else:
            if len(self.last_name_entry.get()) > 0:
                self.last_name = self.last_name_entry.get().strip().split(" ")
                self.last_name = " ".join(i.capitalize() for i in self.last_name)
            else:
                self.last_name = ''
        return (self.first_name, self.last_name)
    
    # Creating a page to display the search results
    def result_function(self,result):
        result_window = Tk()
        result_window.title("Search result")
        headers = ["First_Name", "Last_Name", "Gender", "Date_of_Birth", "Mobile", "Aadhar_Number", "PAN_Number", "Address"]
        total_rows = len(result)
        total_columns = len(headers)
        for i in range(total_columns):
            if i == (total_columns - 1):
                cell = Entry(master = result_window, width = 100)
                cell.grid(row = 0, column = i)
                cell.insert(END, headers[i])
            else:
                cell = Entry(master = result_window, width = 20)
                cell.grid(row = 0, column = i)
                cell.insert(END, headers[i])
        for i in range(1,total_rows+1):
            for j in range(total_columns):
                if j == (total_columns - 1):
                    cell = Entry(master = result_window, width = 100)
                    cell.grid(row = i, column = j)
                    cell.insert(END, result[i-1][j])
                else:
                    cell = Entry(master = result_window, width = 20)
                    cell.grid(row = i, column = j)
                    cell.insert(END, result[i-1][j])
        Button(master = result_window, text = "Ok", command = result_window.destroy).grid(row = total_rows+1, column = 4, sticky = 'news')
        result_window.mainloop()
    
    # Clear button function
    def clear_all(self):
        self.first_name_entry.delete(0, 'end')
        self.last_name_entry.delete(0, 'end')

    # Search button function
    def search(self):
        self.check_data = self.check()
        if len(self.check_data[0]) > 0 and len(self.check_data[1]) > 0:
            mycursor.execute(f"SELECT * FROM applicationDB.Data WHERE First_Name LIKE '%{self.check_data[0]}%' AND Last_Name LIKE '%{self.check_data[1]}%'")
            self.result = [i for i in mycursor]
            if len(self.result) > 0:
                self.clear_all()
                self.result_function(self.result)
            else:
                messagebox.showerror("error", "No records found")
                self.clear_all()
        elif len(self.check_data[0]) > 0 and len(self.check_data[1]) == 0:
            mycursor.execute(f"SELECT * FROM applicationDB.Data WHERE First_Name LIKE '%{self.check_data[0]}%'")
            self.result = [i for i in mycursor]
            if len(self.result) > 0:
                self.clear_all()
                self.result_function(self.result)
            else:
                messagebox.showerror("error", "No records found")
                self.clear_all()
        elif len(self.check_data[1]) > 0 and len(self.check_data[0]) == 0:
            mycursor.execute(f"SELECT * FROM applicationDB.Data WHERE Last_Name LIKE '%{self.check_data[1]}%'")
            self.result = [i for i in mycursor]
            if len(self.result) > 0:
                self.clear_all()
                self.result_function(self.result)
            else:
                messagebox.showerror("error", "No records found")
                self.clear_all()

In [ ]:
# Main page
class app(Tk):
    def __init__(self):
        super().__init__()
        self.title("Main Page")
        self.geometry("300x300")
        self.minsize(300,300)
        self.maxsize(300,300)

        Label(self, text = "Please select an option", font = ("Calibri", 10)).pack()
        Button(self, text = "Insert Data", command = self.open_form).pack(expand = True)
        Button(self, text = "Search Data", command = self.open_search).pack(expand = True)
        Button(self, text = "Close", command = self.destroy).pack(expand = True)
    
    def open_form(self):
        window = insert_window(self)
        window.grab_set()

    def open_search(self):
        window = retrieve_window(self)
        window.grab_set()

In [ ]:
# initializing the App
App = app()
App.mainloop()